# Artificial Neural Network

### Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn

In [ ]:
tf.__version__

'2.11.0'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Part 1 - Data Preprocessing

### Importing the dataset

In [ ]:
dataset = pd.read_csv("/content/Churn_Modelling.csv")
X = dataset.iloc[:,3:-1].values
y = dataset.iloc[:,-1].values

In [ ]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [ ]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

In [ ]:
print(X[:,2])

[0 0 0 ... 0 1 0]


One Hot Encoding the "Geography" column

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
print("France : ", X[0, :3])
print("Spain : ", X[1, :3])
print("Germany : ", X[7, :3])

France :  [1.0 0.0 0.0]
Spain :  [0.0 0.0 1.0]
Germany :  [0.0 1.0 0.0]


### Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [ ]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [ ]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the ANN on the Training set

In [ ]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 3s 5ms/step - loss: 0.6031 - accuracy: 0.7345
Epoch 2/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4947 - accuracy: 0.7986
Epoch 3/100
250/250 [==============================] - 2s 8ms/step - loss: 0.4433 - accuracy: 0.8076
Epoch 4/100
250/250 [==============================] - 2s 8ms/step - loss: 0.4222 - accuracy: 0.8173
Epoch 5/100
250/250 [==============================] - 1s 6ms/step - loss: 0.4107 - accuracy: 0.8205
Epoch 6/100
250/250 [==============================] - 1s 6ms/step - loss: 0.4018 - accuracy: 0.8254
Epoch 7/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3952 - accuracy: 0.8260
Epoch 8/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3898 - accuracy: 0.8280
Epoch 9/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3845 - accuracy: 0.8286
Epoch 10/100
250/250 [==============================] - 2s 6ms/step - loss: 0.3801 - accura

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

**Solution**

In [ ]:
Credit_Score = 600
Geography = 'France'

France = 1
Germany = 0
Spain = 0

Gender = 'Male'
Gender = 1

Age = 40
Tenure = 3
Balance = 60000
Number_of_Products = 2
credit_card = 1
Active_Member = 1
Estimated_Salary = 50000
input_data = np.array([France, Germany, Spain, Credit_Score, Gender, Age, Tenure, Balance, Number_of_Products, credit_card, Active_Member, Estimated_Salary])

In [ ]:
output = ann.predict(sc.transform([input_data]))
print(output)
if output > 0.5:
  print('Yes, Its time to say goodbye')
else:
  print("No, He's our valuable customer")

1/1 [==============================] - 0s 143ms/step
[[0.02452829]]
No, He's our valuable customer


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 3ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
print("Accuracy :", accuracy_score(y_test, y_pred) * 100)

[[1526   69]
 [ 205  200]]
Accuracy : 86.3
